In [ ]:
import pandas as pd
import sqlite3 as sql
import nltk

In [ ]:
conn = sql.connect("track_metadata.db")
# conn2 = sql.connect("dat/mxm_dataset.db")
c = conn.cursor()
for row in c.execute("SELECT name FROM main.sqlite_master WHERE type='table';"):
    print(row)
c.execute("attach 'mxm_dataset.db' AS lyr;")
c.execute("attach 'lastfm_tags.db' AS tag;") 
for row in c.execute("SELECT name FROM lyr.sqlite_master WHERE type='table';"):
    print(row)
for row in c.execute("SELECT name FROM tag.sqlite_master WHERE type='table';"):
    print(row)
q = '''
SELECT * FROM songs
LIMIT 200;
'''
q2 = '''
SELECT * FROM lyrics
LIMIT 200;
'''
c1 = '''
ATTACH DATABASE "dat/mxm_dataset.db" AS conn3;
'''

c2 = '''
ATTACH DATABASE "dat/artist_term.db" AS conn3;
'''

songs = pd.read_sql("SELECT * FROM main.songs", conn)
lyrics = pd.read_sql("SELECT * FROM lyr.lyrics LIMIT 100", conn)
genres = pd.read_sql("SELECT * FROM tag.tags LIMIT 100", conn)
gtags = pd.read_sql("SELECT * FROM tag.tid_tag LIMIT 100", conn)

# Cleaning Begins!

In [ ]:
#artist name, song title, release, all lowercase
songs["title"] = songs["title"].str.lower()
songs["release"] = songs["release"].str.lower()
songs["artist_name"] = songs["artist_name"].str.lower()

#drops artist hotttness metric; drops na vals
songs = songs.drop(columns = ["artist_hotttnesss", "track_7digitalid", "shs_perf", "shs_work", "artist_mbid"])

songs = songs.dropna()

#trim songs with years before 1975
songs = songs[songs.year > 1975]

#renaming the title column to "song_name" and rename "release" to "album_name"
songs = songs.rename(columns={"title":"song_name", "release":"album_name"})

In [1]:
#dropping based on familiarity being below 20
songs = songs[songs.artist_familiarity > .2]
songs = songs.sort_values(by = "artist_familiarity", ascending = False)

#removes rows with blank song entries
songs = songs[songs.song_name.str.len() > 1].sort_values(by="song_name")

#removes rows with years before 1975
songs = songs[songs.year > 1975]

#duplicate removal--removes all duplicate songs by matching artist id
songs = songs.sort_values(by = "artist_name")
songs["s/a"] = songs["song_name"]+songs["artist_id"] #song-artist concatenation column
songs = songs.drop_duplicates("s/a", keep = 'first')

NameError: name 'songs' is not defined

 # TODO: if alternate versions of songs exist, keep the original only

In [ ]:
#IMPLIMENT IRENE!

# Begin Merging Genre and Lyrics Section

In [2]:
#this section is to determine the most popular genre labels

"""sg = pd.read_sql("SELECT tag.tids.tid, tag.tags.tag FROM tag.tid_tag \
            INNER JOIN tag.tags ON tag.tags.ROWID==tag.tid_tag.tag \
            INNER JOIN tag.tids ON tag.tid_tag.tid==tag.tids.ROWID \
             LIMIT 1000000", conn)"""
sg = pd.read_sql("SELECT tag.tids.tid, tag.tags.tag FROM tag.tid_tag \
            INNER JOIN tag.tags ON tag.tags.ROWID==tag.tid_tag.tag \
            INNER JOIN tag.tids ON tag.tid_tag.tid==tag.tids.ROWID \
             ", conn)

data = pd.DataFrame()
sg['tag'] = sg.tag.str.lower()
gdict = sg['tag'].value_counts().to_dict()
populargenres = pd.DataFrame.from_dict(gdict, orient='index', columns = ['genre_count'])

NameError: name 'pd' is not defined